In [ ]:
%matplotlib inline
import pandas as pd
import os
import scipy.sparse
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fp = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'

In [ ]:
case_to_article_links = pd.read_csv(os.path.join(fp, 'case_to_article_title_links.csv'))

In [ ]:
case_to_article_links.shape

In [ ]:
case_to_article_links.head()

In [ ]:
# Select only one entry for each source,target combination
case_to_article_links_distinct = case_to_article_links.groupby(['source', 'target']).nunique()['source']

In [ ]:
# Store the order of the identifiers
case_ids = case_to_article_links_distinct.index.levels[0]
article_ids = case_to_article_links_distinct.index.levels[1]

In [ ]:
len(case_ids), len(article_ids)

In [ ]:
# put edgelist in sparse matrix format
mat_sparse = scipy.sparse.coo_matrix(
                (case_to_article_links_distinct.values, 
                 (case_to_article_links_distinct.index.labels[0], case_to_article_links_distinct.index.labels[1])))

In [ ]:
mat_sparse

In [ ]:
# the cocitation matrix is simply the dot product
mat_cocitation = mat_sparse.T.dot(mat_sparse)

In [ ]:
mat_cocitation

In [ ]:
# retrieve indices of non-zero entries
x_index, y_index, data = scipy.sparse.find(mat_cocitation)

In [ ]:
df_cocitation = pd.DataFrame({'source':article_ids[x_index],
             'target': article_ids[y_index],
             'weight': data})

In [ ]:
# Drop self-loops and duplicates
df_cocitation = df_cocitation[df_cocitation['source']<df_cocitation['target']]

In [ ]:
df_cocitation.index.name = 'id'

In [ ]:
df_cocitation.shape

In [ ]:
# Write to database
import sqlalchemy
engine = sqlalchemy.create_engine('mysql+pymysql://dafne@localhost/caselaw?charset=utf8')
df_cocitation.to_sql('links_article_to_article_title', engine)

In [ ]:
df_cocitation.to_csv(os.path.join(fp, 'article_to_article_title.csv'))

In [ ]:
df_cocitation.weight.max()

In [ ]:
# Look at the weight distribution
df_cocitation['weight'].hist(bins=range(50));

In [ ]:
# What part of the netwerk remains if we cutoff beyond a certain weight value?
df_cocitation['weight'].hist(bins=range(20), cumulative=True, histtype='step', normed=1);

In [ ]:
# How many nodes do we have?
len(set(df_cocitation['source'].unique()).union(set(df_cocitation['target'].unique())))

In [ ]:
# How large is the network (weights and edges) for each cut-off value?
w_max = 20

n = np.zeros(w_max)
m = np.zeros(w_max)

for i in range(w_max):
    df_sub =  df_cocitation[df_cocitation['weight']>=i+1]
    n[i] = len(set(df_sub['source'].unique()).union(set(df_sub['target'].unique())))
    m[i] = len(df_sub)

In [ ]:
plt.plot(np.arange(1, w_max+1), n, label='# nodes')
plt.plot(np.arange(1, w_max+1), m, label='# edges')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
plt.plot(np.arange(1, w_max+1), n, label='# nodes')
plt.plot(np.arange(1, w_max+1), m, label='# edges')
plt.legend()
plt.show()

In [ ]:
df_min5 =  df_cocitation[df_cocitation['weight']>=5]

In [ ]:
df_min5.to_csv(os.path.join(fp, 'article_to_article_title_min5.csv'))

In [ ]:
df_min5.sort_values('weight', ascending=False).head()

In [ ]:
article_nodes = pd.read_csv(os.path.join(fp, 'article_nodes_nodup.csv'), encoding='utf-8')

In [ ]:
article_nodes['book'] = article_nodes.title.str.split(',').map(lambda l: l[0])

In [ ]:
node_ids = set(df_min5['source'].unique()).union(set(df_min5['target'].unique()))

In [ ]:
nodes_min5 = article_nodes[article_nodes.id.isin(node_ids)].copy()
nodes_min5.shape

## Network statistics

In [ ]:
import networkx as nx
import community

In [ ]:
graph = nx.from_pandas_dataframe(df_min5, 'source', 'target', edge_attr=True)

In [ ]:
statistics = {
    'degree': graph.degree(),
    'degree_centrality': nx.degree_centrality(graph),
    'betweenness_centrality': nx.betweenness_centrality(graph),
    'closeness_centrality': nx.closeness_centrality(graph),
}

In [ ]:
partition = community.best_partition(graph)

In [ ]:
modularity = community.modularity(partition, graph)

In [ ]:
nodes_min5['community'] = [str(partition[n_id]) for n_id in nodes_min5['id']]

In [ ]:
c_tab = nodes_min5.groupby(['community', 'authority']).count()['id'].unstack().fillna(0)
c_tab

In [ ]:
nodes_min5.to_csv(os.path.join(fp, 'article_nodes_nodup_min5.csv'), index=False)